In [28]:
%pip install kagglehub
%

Note: you may need to restart the kernel to use updated packages.


In [5]:
import kagglehub
import pandas as pd

In [8]:
# Download latest version of data
path = kagglehub.dataset_download("naserabdullahalam/phishing-email-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/trevorchartier/.cache/kagglehub/datasets/naserabdullahalam/phishing-email-dataset/versions/1


In [10]:
data = pd.read_csv(path +"/phishing_email.csv")
data.head()

,text_combined,label
0,hpl nom may 25 2001 see attached file hplno 52...,0
1,nom actual vols 24 th forwarded sabrae zajac h...,0
2,enron actuals march 30 april 1 201 estimated a...,0
3,hpl nom may 30 2001 see attached file hplno 53...,0
4,hpl nom june 1 2001 see attached file hplno 60...,0


In [25]:
print("Shape of Data:", data.shape)
print("Number of phishing instances: ", sum(data['label'] == 1))
print("Number of legitimate instances: ", sum(data['label'] == 0))

Shape of Data: (82486, 2)
Number of phishing instances:  42891
Number of legitimate instances:  39595


In [27]:
pd.isna(data).any()

text_combined    False
label            False
dtype: bool